# Predicting League of Legends Matches via PyTorch

In [1]:
#env
import jovian
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, TensorDataset, DataLoader

In [2]:
proj_name = 'lol_nn'

---

In [3]:
df = pd.read_csv('high_diamond_ranked_10min.csv') #df = pd.read_csv('/notebooks/storage/high_diamond_ranked_10min.csv')


In [4]:
df.drop(['gameId','redFirstBlood','blueTotalGold','redTotalGold','blueTotalExperience','redTotalExperience','redGoldDiff','redExperienceDiff','redKills','redDeaths'], axis=1, inplace=True)

In [5]:
df.head()

,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,...,redAssists,redEliteMonsters,redDragons,redHeralds,redTowersDestroyed,redAvgLevel,redTotalMinionsKilled,redTotalJungleMinionsKilled,redCSPerMin,redGoldPerMin
0,0,28,2,1,9,6,11,0,0,0,...,8,0,0,0,0,6.8,197,55,19.7,1656.7
1,0,12,1,0,5,5,5,0,0,0,...,2,2,1,1,1,6.8,240,52,24.0,1762.0
2,0,15,0,0,7,11,4,1,1,0,...,14,0,0,0,0,6.8,203,28,20.3,1728.5
3,0,43,1,0,4,5,5,1,0,1,...,10,0,0,0,0,7.0,235,47,23.5,1647.8
4,0,75,4,0,6,6,6,0,0,0,...,7,1,1,0,0,7.0,225,67,22.5,1740.4


In [6]:
targets = df[['blueWins']].values.flatten()
features = df.drop('blueWins', axis=1).values

In [7]:
test_size = int(.10 * 9879) # represents size of validation set
val_size = test_size
train_size = 9879 - test_size*2
train_size , val_size, test_size

(7905, 987, 987)

In [8]:
dataset = TensorDataset(torch.tensor(features).float(), torch.tensor(targets))
train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

In [9]:
batch_size = 128

In [10]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size*2, pin_memory=True)

In [11]:
for x,y in train_loader:
    print(x.shape, y.shape)
    break

torch.Size([128, 29]) torch.Size([128])


In [12]:
input_size = 29
output_size = 2

#### ^^^ what we've done in [part one](https://jovian.ml/richardso21/lol-logistic) ^^^

---

## The Model & Defining The Training Loop

In [13]:
class LOLModelmk2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        return x

In [14]:
crit = F.cross_entropy #criterion
opt_func = torch.optim.SGD #optimizer function (w/o params or lr)

In [15]:
def fit(epochs, lr, model, train_loader, val_loader):
    h = []
    # define optimizer
    opt = opt_func(model.parameters(), lr=lr)
    # loop for num of epochs
    for epoch in range(epochs):
        # training per epoch (iterate tru each batch)
        for inputs, labels in train_loader:
            # put inputs to gpu (explained later)
            inputs, labels = inputs.to(device), labels.to(device)
            # using optimizer & loss
            opt.zero_grad()
            _, loss = step(inputs, labels)
            loss.backward()
            opt.step()
        # evaluate model on validation set every epoch
        val_results = evaluate(model, val_loader)
        # printing as output every 5 epochs
        if (epoch + 1) % 5 == 0 or (epoch + 1) == epochs:
            print(f'Epoch #{epoch + 1} ==> Val Loss: {val_results["avg_loss"]} | Val Acc: {val_results["avg_acc"]}')
        h.append(val_results)
    return h
        
def evaluate(model, loader):
    losses = []
    accs = []
    # tracking gradient not needed
    with torch.no_grad():
        # looping over data loader
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outs, loss = step(inputs, labels, evaluate=True)
            # computing accuracy (function below)
            acc = accuracy(outs, labels)
            losses.append(loss)
            accs.append(acc)
    # avg loss + acc for all data on loader
    avg_loss = sum(losses) / len(losses)
    avg_acc = sum(accs) / len(accs)
    return {'avg_loss':avg_loss, 'avg_acc':avg_acc}
            
# function to input features into model (used for training + validation)
def step(inputs, labels, evaluate=False):
    if evaluate:
        model.eval()
    else:
        model.train()
    outs = model(inputs)
    loss = crit(outs, labels)
    return outs, loss

def accuracy(outs, labels):
    # find the highest probability of the two categories
    _, preds = torch.max(outs, dim=1)
    # return % of correct predictions (matched w/ labels)
    return (torch.tensor(torch.sum(preds==labels).item() / len(preds))) * 100    

In [16]:
def visualize(hist, acc=False):
    losses = [x['avg_loss'] for x in hist]
    accs = [x['avg_acc'] for x in hist]
    if acc:
        plt.plot(accs)
        plt.ylabel('Accuracy (%)')
        plt.title('Accuracy over Epochs')
    else:
        plt.plot(losses)
        plt.ylabel('Losses')
        plt.title('Losses over Epochs')
    plt.xlabel('Epochs')
    plt.show()

---

## Using the GPU

In [17]:
torch.cuda.is_available()

True

In [18]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [19]:
device

device(type='cuda')

In [20]:
model = LOLModelmk2().to(device)

In [21]:
model

LOLModelmk2(
  (fc1): Linear(in_features=29, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)

---

## Training

In [22]:
before_train = evaluate(model, test_loader)
before_train

{'avg_loss': tensor(14.2979, device='cuda:0'), 'avg_acc': tensor(51.2455)}

In [23]:
hist = [evaluate(model, val_loader)]

In [ ]:
hist += fit(25, 1e-4, model, train_loader, val_loader)

Epoch #5 ==> Val Loss: 0.6475211381912231 | Val Acc: 70.31160736083984
Epoch #10 ==> Val Loss: 0.5950438976287842 | Val Acc: 70.65274047851562
Epoch #15 ==> Val Loss: 0.623634934425354 | Val Acc: 69.88798522949219


In [ ]:
hist += fit(50, 1e-5, model, train_loader, val_loader)

In [ ]:
after_train = evaluate(model, test_loader)
after_train

In [ ]:
visualize(hist)

In [ ]:
visualize(hist, acc=True)

---

In [ ]:
class LOLModelmk3(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, output_size)
        
        self.dp = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dp(x)
        x = F.relu(self.fc2(x))
        x = self.dp(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
model = LOLModelmk3().to(device)

In [ ]:
model

In [ ]:
before_train = evaluate(model, test_loader)
before_train

In [ ]:
hist = [evaluate(model, val_loader)]

In [ ]:
hist += fit(25, 1e-4, model, train_loader, val_loader)

In [ ]:
hist += fit(50, 1e-5, model, train_loader, val_loader)

In [ ]:
after_train = evaluate(model, test_loader)
after_train

In [ ]:
visualize(hist)

In [ ]:
visualize(hist, acc=True)

---

In [ ]:
class LOLModelmk4(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.bn1 = nn.BatchNorm1d(num_features=64)
        self.fc2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(num_features=32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, output_size)
        
        self.dp = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dp(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dp(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
model = LOLModelmk4().to(device)

In [ ]:
before_train = evaluate(model, test_loader)
before_train

In [ ]:
hist = [evaluate(model, val_loader)]

In [ ]:
hist += fit(25, 1e-3, model, train_loader, val_loader)

In [ ]:
hist += fit(50, 1e-4, model, train_loader, val_loader)

In [ ]:
after_train = evaluate(model, test_loader)
after_train

In [ ]:
visualize(hist)

In [ ]:
visualize(hist, acc=True)

In [ ]:
jovian.commit(filename=proj_name, project=proj_name, environment=None)